# Decorators

In [45]:
global_counter_dic = {}

# DEFINE COUNTER DECORATOR
from functools import wraps
def counter(fn):
    global_counter_dic[fn.__name__] = 0
    @wraps(fn)  # Make sure help(fn) returns meaninful info
    def inner(*args, **kwargs):
        # Not using nonlocal c
        #nonlocal c
        print("a")
        global_counter_dic[fn.__name__] += 1
        return fn(*args, **kwargs)
    
    return inner

# Apply decorator directly during the definition of my function
@counter
def plus(a, b=0):
    """
    Additionne deux nombres
    """
    return a+b
# same as:
#plus = counter(plus)


r = plus(1,2)
print("r = ", r)
r = plus(1,100)
print("r = ", r)
help(plus)
print (global_counter_dic)


a
r =  3
a
r =  101
Help on function plus in module __main__:

plus(a, b=0)
    Additionne deux nombres

{'plus': 2}


# Timer decorator
We could decorate with a timer that call the function multiple times to get averages...
Question is: do you want to decorate with a timer once to benchmark, or you want to always have timers attaches to some of your function

In [2]:
# Timer decorator
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        el = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        
        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__, args_str, el))
        
        return result
    return inner

In [8]:
#@timed
def fib_rec(n):
    if n <= 1:
        return 1
    else:
        return fib_rec(n-1) + fib_rec(n-2)

    
@timed
def fib_rec_wrap(n):
    return fib_rec(n)
print(fib_rec_wrap(30))

@timed
def fib_loop(n):
    fib1 = 1
    fib2 = 2
    for i in range(3, n+1):
        fib1, fib2 = fib2, fib1 + fib2
    return fib2

print(fib_loop(30))

from functools import reduce
@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, n: (prev[0] + prev[1], prev[0]), dummy, initial)
    return fib_n[0]
                   
print(fib_reduce(30))
                                                              

fib_rec_wrap(30) took 0.182585s to run
1346269
fib_loop(30) took 0.000003s to run
1346269
fib_reduce(30) took 0.000004s to run
1346269


# Memoization decorator

In [17]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}
        
    def fib(self, n):
        if n not in self.cache:
            print("Calculating Fib of ", n)
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]
    

In [18]:
f = Fib()
f.fib(10)

Calculating Fib of  10
Calculating Fib of  9
Calculating Fib of  8
Calculating Fib of  7
Calculating Fib of  6
Calculating Fib of  5
Calculating Fib of  4
Calculating Fib of  3


55

In [19]:
f.fib(10)

55

In [20]:
# We can do the same caching with a closure

In [30]:
# Memoization generic decorator
# TBD: Force fn to have only one parameter
# TBD: Cache limit and LRU algo...
def memoize(fn):
    cache = dict() # M
    
    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

In [31]:
@memoize
def fact(n):
    print("Fact of ", n)
    return 1 if n < 2 else n * fact(n-1)


In [32]:
fact(10)

Fact of  10
Fact of  9
Fact of  8
Fact of  7
Fact of  6
Fact of  5
Fact of  4
Fact of  3
Fact of  2
Fact of  1


3628800

In [33]:
fact(10)

3628800

In [34]:
help(fact)

Help on function fact in module __main__:

fact(n)



In [35]:
# Built-in lru_cache
# default cache size is around 128 entries
from functools import lru_cache
@lru_cache
def fib(n):
    print("calc fib of ", n)
    return 1 if n < 3 else fib(n-1) + fib(n=2)

In [36]:
fib(11)

calc fib of  11
calc fib of  10
calc fib of  9
calc fib of  8
calc fib of  7
calc fib of  6
calc fib of  5
calc fib of  4
calc fib of  3
calc fib of  2
calc fib of  2


10

In [37]:
fib(11)

10

# Decorators Part 2
### Decorator Factory

In [61]:
# Time-bench decorator

def time_bench(reps):
    def time_bench_dec(fn): # DECORATOR CAN ONLY TAKE ONE PARAMETER: THE FUNCTION TO DECORATE
        from time import perf_counter
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            total_el = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_el += perf_counter() - start
            avg = total_el / reps  # Using the free variable "reps"

            args_ = [str(a) for a in args]
            kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
            all_args = args_ + kwargs_
            args_str = ','.join(all_args)

            print('{0}({1}) took an average of {2:.6f}s to run'.format(fn.__name__, args_str, avg))

            return result
        return inner
    return time_bench_dec  # RETURNING THE DECORATOR

In [64]:
def fib_r(n):
    #print("calc fib of ", n)
    return 1 if n < 3 else fib_r(n-1) + fib_r(n-2)

@time_bench(10)  ## CALLING THE DECORATOR FACTORY
def fib_rs(n):
    return fib_r(n)

fib_rs(30)

fib_rs(30) took an average of 0.132659s to run


832040

# Decorators factory from a callable class

In [65]:
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print("decorated called: ", a, b)
            return fn(*args, **kwargs)
        return inner
    return dec

@my_dec(10, 20)
def my(s):
    print(s)

my("allo")

decorated called:  10 20
allo


In [70]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, c):
        print("called ", self.a, self.b, c)
        
my = MyClass(1, 2)

In [71]:
my(3)

called  1 2 3


In [79]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn): # THIS IS a DECORATOR !
        def inner(*args, **kwargs):
            print("decorated called: ", self.a, self.b)
            return fn(*args, **kwargs)
        return inner
        


In [80]:
@MyClass(10, 20)
def my2(s):
    print(s)

In [81]:
my2("allo")

decorated called:  10 20
allo


# Decorating classes

In [87]:
# "Monkey patching"
from fractions import Fraction
Fraction.speak = lambda self, msg: "Msg: " + msg

f = Fraction(2, 3)
f.speak("allo")

'Msg: allo'

In [103]:
def dec_speak2(cls):
    cls.speak2 = lambda self, msg: "Msg: " + msg
    return cls

#@dec_speak2
class MyClass3:
    pass

MyClass3 = dec_speak2(MyClass3)


In [104]:
m = MyClass3()
m.speak2("allo")

'Msg: allo'

# Single Dispatch Generic Functions

In [1]:
from html import escape

In [7]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(items))
             for itm in l
            )
    return ('<ul>\n' + '\n'.join(items) + '\n</ul>')

def html_dict(d):
    items = ('<li{0}={1}</li>'.format(k, v)
             for k, v in d.items())
    return '<ul>' + '@n'.join(items) + '\n</ul>'

In [9]:
print(html_str("""sss
ttt""""))

SyntaxError: unterminated string literal (detected at line 2) (1880449764.py, line 2)

In [10]:
print(html_int(255))

255(<i>0xff</i>)
